In [28]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [29]:
def ASBM(n,k):
    A = np.zeros((n,n))
    #np.random.seed(1)
    for i in range(0,n):
        for j in range(0,n):
            if y[i] == y[j]:
                A[i,j] = np.random.binomial(1, 0.13)
            else:
                A[i,j] = np.random.binomial(1, 0.1)
    for i in range(0,n):
        for j in range(i,n):
            A[i,j]=A[j,i]
    return A

In [30]:
def gety(n):
    y = np.zeros((n,1))
    #np.random.seed(2)
    for i in range(0,n):
        y[i,0] = np.random.choice(np.arange(1, 4), p=[0.2, 0.3,0.5])
    return(y)

In [31]:
def EncoderLDA(A):
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
#ASE*LDA
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    newx=([])
    d=3
    newx=np.concatenate(U[:,0:d]*s[0:d]**0.5)
    newx=np.reshape(newx,(n,d))
    clf2 = LinearDiscriminantAnalysis()
    #print(np.shape(newx))
    newx_train, newx_test =newx[train_index], newx[test_index]
    clf2.fit(newx_train, y_train.ravel())
    ASELDAACC=clf2.score(newx_test, y_test.ravel())
    #time.sleep(2) 
    end1 = time.time()
    ASELDATIME=end1-begin1
    eigenvalues=s**2
    Top=eigenvalues[:10]
#nk,w,Z
    begin5 = time.time()
    global k
    nk = np.zeros((1,k))
    for i in range(0,len(y_train)-1):
        nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
    w = np.zeros((int(np.size(y_train)),k))
    for i in range(0,int(np.size(y_train))):
        k=int(y_train[i])
        w[i][k-1]=1/nk[0,k-1]
    z_train=np.matmul(X_train,w)
    z_test = np.matmul(X_test,w)
    #time.sleep(5)
    end5 = time.time()
    zTIME=end5 - begin5
#AEE*KNN
    begin2 = time.time()
    neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
    neigh.fit(z_train, y_train.ravel())
    AEEKNNACC = neigh.score(z_test, y_test)
    #time.sleep(3) 
    end2 = time.time()
    AEEKNNTIME=end2-begin2
#AEE*LDA
    begin3 = time.time()
    clf4 = LinearDiscriminantAnalysis()
    clf4.fit(z_train,y_train.ravel())
    AEELDAACC = clf4.score(z_test, y_test.ravel())
    #time.sleep(4) 
    end3 = time.time()
    AEELDATIME=end3-begin3
#Get result
    Acc=np.array([ASELDAACC,AEEKNNACC,AEELDAACC])
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    print(f"When n = {n}")
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")

In [33]:
for n in range (100,3100,100):
    k=3
    y=gety(n)
    A=ASBM(n,k=k)
    EncoderLDA(A)

When n = 100
Accuracy for each method: [0.25 0.35 0.5 ]
Time for each method: [0.00455809 0.00428295 0.00221086]
When n = 200
Accuracy for each method: [0.5   0.5   0.625]
Time for each method: [0.01439309 0.00620413 0.00256801]
When n = 300
Accuracy for each method: [0.56666667 0.66666667 0.73333333]
Time for each method: [0.03084898 0.00779486 0.00350785]
When n = 400
Accuracy for each method: [0.4875 0.675  0.75  ]
Time for each method: [0.06300521 0.00760031 0.00345016]
When n = 500
Accuracy for each method: [0.57 0.64 0.74]
Time for each method: [0.0962038  0.0093751  0.00362515]
When n = 600
Accuracy for each method: [0.56666667 0.71666667 0.70833333]
Time for each method: [0.15386605 0.01013517 0.00349307]
When n = 700
Accuracy for each method: [0.51428571 0.72142857 0.72142857]
Time for each method: [0.25536895 0.01047897 0.00503826]
When n = 800
Accuracy for each method: [0.575  0.7875 0.8125]
Time for each method: [0.40521908 0.01510406 0.00529408]
When n = 900
Accuracy for e